In [1]:
import pandas as pd

import json
import glob
import os

In [2]:
base = "/amd.home/home/i11/zeitz/experiments/chpot/"
paths = glob.glob(base + "building_blocks/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{ 
    'features': run['feature_flags'].replace('DEFAULT, TDCCH_APPROX, TDCCH_POSTCUSTOMIZATION, TDCCH_PRECUSTOMIZATION, TDCCH_QUERY_ASTAR, TDCCH_TRIANGLE_SORTING', ''), 
    **algo } 
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') == 'CH Potentials Query'])

In [4]:
queries.columns

Index(['features', 'algo', 'from', 'lower_bound', 'num_pot_evals',
       'num_queue_pops', 'num_relaxed_arcs', 'result', 'running_time_ms',
       'to'],
      dtype='object')

In [5]:
table = queries.groupby('features')[['running_time_ms', 'num_queue_pops', 'num_relaxed_arcs']].mean()
table['num_relaxed_arcs'] = table['num_relaxed_arcs'] / 1000000
table['num_queue_pops'] = table['num_queue_pops'] / 1000
print(table.to_latex())
table

\begin{tabular}{lrrr}
\toprule
{} &  running\_time\_ms &  num\_queue\_pops &  num\_relaxed\_arcs \\
features                                           &                  &                 &                   \\
\midrule
                                                   &       264.243967 &      404.747802 &          2.025657 \\
CHPOT\_NO\_DEG2, CHPOT\_NO\_DEG3,                      &       385.966340 &      822.944687 &          2.278133 \\
CHPOT\_NO\_DEG2, CHPOT\_NO\_DEG3, CHPOT\_NO\_SCC,        &       433.076718 &      914.793945 &          2.525061 \\
CHPOT\_NO\_DEG2, CHPOT\_NO\_DEG3, CHPOT\_NO\_SCC, CHP... &      1370.172339 &     6947.207974 &         19.232052 \\
CHPOT\_NO\_DEG2, CHPOT\_NO\_DEG3, CHPOT\_ONLY\_TOPO,     &      1161.183556 &     6126.160896 &         17.031770 \\
CHPOT\_NO\_DEG3,                                     &       332.243265 &      697.600302 &          2.187592 \\
CHPOT\_NO\_DEG3, CHPOT\_ONLY\_TOPO,                    &      1046.580869 &     5130.09631

,running_time_ms,num_queue_pops,num_relaxed_arcs
features,,,
,264.243967,404.747802,2.025657
"CHPOT_NO_DEG2, CHPOT_NO_DEG3,",385.966340,822.944687,2.278133
"CHPOT_NO_DEG2, CHPOT_NO_DEG3, CHPOT_NO_SCC,",433.076718,914.793945,2.525061
"CHPOT_NO_DEG2, CHPOT_NO_DEG3, CHPOT_NO_SCC, CHPOT_ONLY_TOPO,",1370.172339,6947.207974,19.232052
"CHPOT_NO_DEG2, CHPOT_NO_DEG3, CHPOT_ONLY_TOPO,",1161.183556,6126.160896,17.031770
"CHPOT_NO_DEG3,",332.243265,697.600302,2.187592
"CHPOT_NO_DEG3, CHPOT_ONLY_TOPO,",1046.580869,5130.096317,16.240890
"CHPOT_ONLY_TOPO,",810.817544,2975.816681,14.852560
